<a href="https://colab.research.google.com/github/GoldPapaya/info256-applied-nlp/blob/main/4.topics/HW4_ExploreTopics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dbamman/anlp25/blob/main/4.topics/HW4_ExploreTopics.ipynb)

**N.B.** Once it's open on Colab, remember to save a copy (by e.g. clicking `Copy to Drive` above).

---

# HW4: Exploring topics
In this homework, you have an open-ended task: tell us something **interesting** about a collection of text data using topic modeling. You can choose between the following datasets:

In `acl.all.tsv` you'll find 7,188 papers published at major NLP venues (ACL, EMNLP, NAACL, TACL, etc.) between 2013 and 2020.  Here is a sample of that data:

|id|year of publication|title|abstract|
|---|---|---|---|
|pimentel-etal-2020-phonotactic|2020|Phonotactic Complexity and Its Trade-offs|We present methods for calculating a measure of phonotactic complexity ...|
|wang-etal-2020-amr|2020|AMR-To-Text Generation with Graph Transformer|Abstract meaning representation (AMR)-to-text generation is the challenging task  ...|


In [ ]:
!wget https://raw.githubusercontent.com/dbamman/anlp25/refs/heads/main/data/acl.all.tsv

In `gutenberg.genre.tsv` you'll find 1,250 passages of English-language fiction from Project Gutenberg, categorized by genre (adventure, detective, love stories, science fiction, westerns).  Here's a sample:

|id|genre|author|passage|
|---|---|---|---|
|66390_1796	|love stories	|Prime-Stevenson, Edward	|Only a few days absence. I shall think of you. Imre. P. S. Please write me." I was amused,  ...|
|50157_2780	|detective and mystery stories|	Wheeler, Janet D.	|Edina shook her head. “They think I’ve lied to them. They think I’ve cheated them. They want their money, and you can’t rightly blame them...|


In [ ]:
!wget https://raw.githubusercontent.com/dbamman/anlp25/refs/heads/main/data/gutenberg.genre.tsv

In `convote.train.tsv`, you'll find 2,723 dialogue turns from the Convote dataset on congressional speeches, paired with the party affiliation of the speaker (Democrat/Republican). Here's a sample:

|party|passage|
|---|---|
|R	|mr. speaker , i claim the time in opposition to the motion to recommit .|
|D	|mr. speaker , on that i demand the yeas and nays .|

In [134]:
!wget https://raw.githubusercontent.com/dbamman/anlp25/refs/heads/main/data/convote.train.tsv

--2025-09-20 18:37:34--  https://raw.githubusercontent.com/dbamman/anlp25/refs/heads/main/data/convote.train.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4660140 (4.4M) [text/plain]
Saving to: ‘convote.train.tsv’

convote.train.tsv   100%[===================>]   4.44M  --.-KB/s    in 0.09s   

2025-09-20 18:37:34 (51.8 MB/s) - ‘convote.train.tsv’ saved [4660140/4660140]



Choose one of these datasets and use topic modeling as exploratory data analysis to find some interesting structure in that data.  There are only two constraints:

* You must use topic modeling to find topics in the data
* You must relate those **topics** to some aspect of the metadata. Examples of this could be: charting the rise and fall of **topics** over time (i.e., with year of publication as metadata) or associating **topics** with genre to find which topics are more aligned one one genre over another (perhaps using some of the measures of association we talked about earlier this semester).

## Part 1: topic modeling

Q1: Try creating several topic models with different parameters for the _number_ of topics. Compare the outputs, and decide what number of topics is most reasonable for your dataset. **In a couple of sentences, justify your selection.**

In [141]:
# The next bit of code is copied from TopicModel.ipynb for stopword
# filtering

!pip install gensim

In [142]:
import operator
import re

import nltk
from gensim import corpora
from tqdm import tqdm  # for progress bars

nltk.download("stopwords")
nltk.download("punkt")
nltk.download("punkt_tab")
import random

import numpy as np
import pandas as pd
from nltk.corpus import stopwords

random.seed(1)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [143]:
!wget https://raw.githubusercontent.com/dbamman/anlp25/main/data/jockers.stopwords

--2025-09-20 18:43:16--  https://raw.githubusercontent.com/dbamman/anlp25/main/data/jockers.stopwords
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 44453 (43K) [text/plain]
Saving to: ‘jockers.stopwords.4’

jockers.stopwords.4 100%[===================>]  43.41K  --.-KB/s    in 0.01s   

2025-09-20 18:43:17 (4.03 MB/s) - ‘jockers.stopwords.4’ saved [44453/44453]



In [144]:
def read_stopwords(filename):
    """Reads a file of stopwords into a set."""
    stopwords = set([
        line.rstrip() for line in open(filename)
    ])
    return stopwords

In [145]:
stop_words = set(stopwords.words('english'))
stop_words = stop_words | read_stopwords("jockers.stopwords")
stop_words.add("'s")
stop_words=list(stop_words)

In [146]:
pattern = re.compile(r"[A-Za-z]")
def stopword_filter(word, stopwords):
    """ Function to exclude words from a text."""

    # no stopwords
    if word in stopwords:
        return False

    # has to contain at least one letter
    if pattern.search(word) is not None:
        return True

    return False

In [150]:
df = pd.read_csv('convote.train.tsv', sep='\t', names=['party', 'passage'])
df.head(10)

,party,passage
0,D,"mr. speaker , i rise in opposition to the rule..."
1,D,"mr. speaker , i rise for a constitutional poin..."
2,D,"mr. speaker , the resolution we are preparing ..."
3,D,"mr. speaker , consistent with the oath of offi..."
4,D,"mr. speaker , on that i demand the yeas and na..."
5,R,"mr. speaker , i rise in strong support of this..."
6,D,"mr. speaker , the chairman of the committee on..."
7,D,"mr. speaker , we are making a terrible mistake..."
8,R,"mr. speaker , i offer a privileged resolution ..."
9,R,"mr. speaker , for the purposes of debate only ..."


In [151]:
from nltk import word_tokenize

def preprocess(text, stop_words):
    tokens = word_tokenize(text.lower())
    tokens = [word for word in tokens if stopword_filter(word, stop_words)]
    return tokens

tokenized_passages = [preprocess(passage, stop_words) for passage in df['passage']]

[['mr.', 'speaker', 'rise', 'opposition', 'rules', 'package', 'today', 'outrageous', 'republican', 'colleagues', 'placed', 'rules', 'package', 'lacks', 'integrity', 'worst', 'completely', 'unethical', 'highest', 'body', 'elected', 'officials', 'country', 'held', 'highest', 'ethical', 'standards', 'republican', 'colleagues', 'opted', 'rules', 'package', 'lowers', 'ethics', 'standards', 'promote', 'agenda', 'cost', 'rules', 'package', 'difficult', 'ethics', 'investigations', 'take', 'place', 'requiring', 'majority', 'ethics', 'committee', 'investigation', 'danger', 'diminishing', 'integrity', 'institution', 'rule', 'majority', 'party', 'effectively', 'block', 'ethics', 'investigation', 'member', 'party', 'abuse', 'power', 'democrats', 'oppose', 'plan', 'americans', 'country', 'expressed', 'opposition', 'plan', 'democratic', 'colleagues', 'plan', 'strengthen', 'ethics', 'rules', 'improve', 'congressional', 'accountability', 'sure', 'legislation', 'properly', 'considered', 'republican', 'p

In [152]:
dictionary = corpora.Dictionary(tokenized_passages)
dictionary.filter_extremes(no_below=5, no_above=.5, keep_n=10000)
corpus = [dictionary.doc2bow(text) for text in tokenized_passages]

In [153]:
import gensim

lda_model20 = gensim.models.ldamodel.LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=20,
    passes=10,
    alpha='auto'
)

lda_model10 = gensim.models.ldamodel.LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=10,
    passes=10,
    alpha='auto'
)

lda_model5 = gensim.models.ldamodel.LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=5,
    passes=10,
    alpha='auto'
)

In [155]:
for i in range(20):
    print("topic %s:\t%s" % (i, ' '.join([term for term, freq in lda_model20.show_topic(i, topn=10)])))
print('\n')
for i in range(10):
    print("topic %s:\t%s" % (i, ' '.join([term for term, freq in lda_model10.show_topic(i, topn=10)])))
print('\n')
for i in range(5):
    print("topic %s:\t%s" % (i, ' '.join([term for term, freq in lda_model5.show_topic(i, topn=10)])))

topic 0:	time gentleman people balance yield reserve want going chairman question
topic 1:	pension plans workers employees small plan benefits benefit employers h.r
topic 2:	people going want house american today congress time republican majority
topic 3:	health care programs education people cuts americans cut children funding
topic 4:	budget billion year security deficit funding spending cuts programs amt
topic 5:	gasoline refinery refineries capacity oil energy prices price country refining
topic 6:	brac air base commission guard defense national military process recommendations
topic 7:	chairman gentleman thank time work committee vote want energy appreciate
topic 8:	committee commission house report resolution security recommendations gouging task force
topic 9:	trade energy oil agreement gas states free companies u.s. united
topic 10:	lawsuits motion recommit house firearms rule vote legislation rules lawyers
topic 11:	security states legislation h.r border federal law driver enf

Q2: Using the topic model with the number of topics that you selected, identify a couple of topics that are coherent. **In a paragraph, interpret these topics (i.e., by giving them a name / description). Support your answer with evidence from word and/or document distributions.**

## Part 2: analysis

Q3: Relate the topics to some aspect of the metadata. These don't have to be the same topics that you described in question 2. **Write a paragraph synthesizing your findings, and provide quantitative evidence.**

---

## To submit

Congratulations on finishing this homework!
Please follow the instructions below to download the notebook file (`.ipynb`) and its printed version (`.pdf`) for submission on bCourses -- remember **all cells must be executed**.

1.  Download a copy of the notebook file: `File > Download > Download .ipynb`.

2.  Print the notebook as PDF (via your browser, or tools like [nbconvert](https://nbconvert.readthedocs.io/en/latest/)).